In [1]:
import pandas as pd

df = pd.read_csv("business_names_with_categories.csv")
df

,name,fsq_category_ids,Count,category_label
0,Western Union,63be6904847c3692a84b9b3d,5993,Business and Professional Services > Financial...
1,The PNC Financial Services Group,4bf58dd8d48988d10a951735,5032,Business and Professional Services > Financial...
2,Starbucks,4bf58dd8d48988d1e0931735,4893,"Dining and Drinking > Cafe, Coffee, and Tea Ho..."
3,Blue Rhino,63be6904847c3692a84b9b34,4818,Business and Professional Services > Chemicals...
4,Redbox,4bf58dd8d48988d126951735,4718,Retail > Video Store
...,...,...,...,...
17467,American Appliance Repair,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...
17468,A-1 Appliance Service,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...
17469,Affordable Appliance Repair,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...
17470,NightLight Pediatric Urgent Care,4bf58dd8d48988d194941735,5,Health and Medicine > Emergency Service > Emer...


In [2]:
df["category_array"] = df["category_label"].str.split(" > ")
df

,name,fsq_category_ids,Count,category_label,category_array
0,Western Union,63be6904847c3692a84b9b3d,5993,Business and Professional Services > Financial...,"[Business and Professional Services, Financial..."
1,The PNC Financial Services Group,4bf58dd8d48988d10a951735,5032,Business and Professional Services > Financial...,"[Business and Professional Services, Financial..."
2,Starbucks,4bf58dd8d48988d1e0931735,4893,"Dining and Drinking > Cafe, Coffee, and Tea Ho...","[Dining and Drinking, Cafe, Coffee, and Tea Ho..."
3,Blue Rhino,63be6904847c3692a84b9b34,4818,Business and Professional Services > Chemicals...,"[Business and Professional Services, Chemicals..."
4,Redbox,4bf58dd8d48988d126951735,4718,Retail > Video Store,"[Retail, Video Store]"
...,...,...,...,...,...
17467,American Appliance Repair,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...,"[Retail, Furniture and Home Store, Home Applia..."
17468,A-1 Appliance Service,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...,"[Retail, Furniture and Home Store, Home Applia..."
17469,Affordable Appliance Repair,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...,"[Retail, Furniture and Home Store, Home Applia..."
17470,NightLight Pediatric Urgent Care,4bf58dd8d48988d194941735,5,Health and Medicine > Emergency Service > Emer...,"[Health and Medicine, Emergency Service, Emerg..."


In [3]:
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:1234/v1", api_key="e")

In [4]:
prompt = """
You are a financial domain expert. You have been tasked with generating a string which is a transaction made to the mentioned business in the mentioned industry/category.

Few examples:

Business Name: SAFEWAY, Industry/Category: Retail/Food and Beverage Retail/Grocery Store, Transaction String: SAFEWAY #1234 Purchase $127.42 on 11/15/25
Business Name: THE CORNER CAFE L, Industry/Category: Dining and Drinking/Cafe, Coffee, and Tea House/Café, Transaction String: SQ *SQ *THE CORNER CAFE L PHOENIX AZ
Business Name: AMZN Mktp US, Industry/Category: Retail/Miscellaneous Store, Transaction String: AMZN Mktp US*A1B2C3D4 $24.99 AMZN.COM/BILL
Business Name: COSTCO GAS, Industry/Category: Travel and Transportation/Fuel Station, Transaction String: COSTCO GAS #567 15.4 GAL @ 3.89 $59.90
Business Name: STARBUCKS COFFEE, Industry/Category: Dining and Drinking/Cafe, Coffee, and Tea House/Coffee Shop, Transaction String: STARBUCKS COFFEE #98765 STORE 98765 PURCHASE $8.75
Business Name: Uber, Industry/Category: Travel and Transportation/Transportation Service/Taxi, Transaction String: Uber*Trip 1800-555-1234 CA $22.50
Business Name: AUDIBLE.COM, Industry/Category: Retail/Bookstore, Transaction String: AUDIBLE.COM/CHARGE 888-841-2453 NJ
Business Name: DELTA AIR LINES, Industry/Category: Travel and Transportation, Transaction String: DELTA AIR LINES TICKET 00621
Business Name: SAFEWAY, Industry/Category: Retail/Food and Beverage Retail/Grocery Store, Transaction String: SAFEWAY #1234
Business Name: THE CORNER CAFE L, Industry/Category: Dining and Drinking/Cafe, Coffee, and Tea House/Café, Transaction String: SQ *SQ *THE CORNER CAFE L, $15.30
Business Name: AMZN Mktp US, Industry/Category: Retail/Miscellaneous Store, Transaction String: AMZN Mktp US*A1B2C3D4 AMZN.COM/BILL WA
Business Name: COSTCO GAS, Industry/Category: Travel and Transportation/Fuel Station, Transaction String: COSTCO GAS #567, $75.00
Business Name: STARBUCKS COFFEE, Industry/Category: Dining and Drinking/Cafe, Coffee, and Tea House/Coffee Shop, Transaction String: STARBUCKS COFFEE #98765 PURCHASE
Business Name: Uber, Industry/Category: Travel and Transportation/Transportation Service/Taxi, Transaction String: Uber*Trip HELP.UBER.COM CA, $45.10
Business Name: AUDIBLE.COM, Industry/Category: Retail/Bookstore, Transaction String: AUDIBLE.COM/CHARGE $14.95
Business Name: DELTA AIR LINES, Industry/Category: Travel and Transportation, Transaction String: DELTA AIR LINES TICKET 00621578934, $450.80, CONF #XYZ123

You may or may not include information such as amount, date, location, etc. Be creative but keep it realistic. 
Always exclude any personally identifiable information such as full card numbers, account numbers, names of individuals, etc.
Now, generate a similar transaction string for the following business:
Business Name: {business_name}, Industry/Category: {category}, Transaction String:
"""

In [11]:
def generate_transaction_string(business_name, category, seed):
    response = client.chat.completions.create(
        model="qwen3-4b-instruct-2507",
        messages=[
            {"role": "system", "content": "You are a financial domain expert. Strictly follow the instructions from the user."},
            {"role": "user", "content": prompt.format(
                business_name=business_name,
                category=category
            )}
        ],
        temperature=2,
        seed=seed,
    )
    string = response.choices[0].message.content.strip()
    print(f"Generated transaction string for {business_name}: {string}")
    return string

In [12]:
from tqdm.notebook import tqdm
import random
tqdm.pandas()

def _gen_transaction(row):
    seed = random.randint(0, 2**31 - 1)
    tx = generate_transaction_string(row["name"], row["category_label"], seed)
    return pd.Series([tx, seed], index=["transaction_string", "seed"])

df[["transaction_string", "seed"]] = df.progress_apply(_gen_transaction, axis=1)

  0%|          | 0/17472 [00:00<?, ?it/s]

Generated transaction string for Western Union: WESTERN UNION #45678 SEND $29.99 TO CHICAGO IL VIA EFT, REF #WU987654321
Generated transaction string for The PNC Financial Services Group: The PNC Financial Services Group #7890 ACCOUNT MOVEMENT $1,250.00 TO CHECKING ACCT 4567-XYZ LOCATION PITTSBURGH PA DATE 03/22/25
Generated transaction string for Starbucks: STARBUCKS COFFEE #98765 STORE 98765 PURCHASE $8.75
Generated transaction string for Blue Rhino: BLUE RHINO #4567 PURCHASE $1,250.00 FOR LABORATORY GAS SUPPLIES IN AZ, ORDER REF 98765-ALPHA
Generated transaction string for Redbox: Redbox #4567 DVD RENTAL $12.99 02/03/25 AZ
Generated transaction string for Subway: SUBWAY #7892 SANDWICH PURCHASE $14.95 PHOENIX AZ
Generated transaction string for Subway: SUBWAY #4567 PURCHASE $12.99 ON 11/18/25 AT DOWNTOWN LOCATION, AZ
Generated transaction string for Electric Charging Station: Electric Charging Station #888 CHARGE 15.2 kWh @ $0.39/kWh $5.93 PHOENIX AZ
Generated transaction string for 

In [13]:
df

,name,fsq_category_ids,Count,category_label,category_array,transaction_string,seed
0,Western Union,63be6904847c3692a84b9b3d,5993,Business and Professional Services > Financial...,"[Business and Professional Services, Financial...",WESTERN UNION #45678 SEND $29.99 TO CHICAGO IL...,1127428596
1,The PNC Financial Services Group,4bf58dd8d48988d10a951735,5032,Business and Professional Services > Financial...,"[Business and Professional Services, Financial...",The PNC Financial Services Group #7890 ACCOUNT...,292935982
2,Starbucks,4bf58dd8d48988d1e0931735,4893,"Dining and Drinking > Cafe, Coffee, and Tea Ho...","[Dining and Drinking, Cafe, Coffee, and Tea Ho...",STARBUCKS COFFEE #98765 STORE 98765 PURCHASE $...,1047754538
3,Blue Rhino,63be6904847c3692a84b9b34,4818,Business and Professional Services > Chemicals...,"[Business and Professional Services, Chemicals...","BLUE RHINO #4567 PURCHASE $1,250.00 FOR LABORA...",1537448773
4,Redbox,4bf58dd8d48988d126951735,4718,Retail > Video Store,"[Retail, Video Store]",Redbox #4567 DVD RENTAL $12.99 02/03/25 AZ,1248492378
...,...,...,...,...,...,...,...
17467,American Appliance Repair,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...,"[Retail, Furniture and Home Store, Home Applia...",AMERICAN APPLIANCE REPAIR #7890 SERVICE CALL 1...,900410143
17468,A-1 Appliance Service,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...,"[Retail, Furniture and Home Store, Home Applia...",A-1 Appliance Service #4567 SERVICE CALL 2025-...,957908134
17469,Affordable Appliance Repair,63be6904847c3692a84b9bf1,5,Retail > Furniture and Home Store > Home Appli...,"[Retail, Furniture and Home Store, Home Applia...",AFFORDABLE APPLIANCE REPAIR #4567 SERVICE CALL...,935033619
17470,NightLight Pediatric Urgent Care,4bf58dd8d48988d194941735,5,Health and Medicine > Emergency Service > Emer...,"[Health and Medicine, Emergency Service, Emerg...",NIGHTLIGHT PEDIATRIC URGENT CARE #789 VISIT 11...,359454038


In [14]:
df.to_csv("business_transactions_dataset.csv", index=False)